In [7]:
"""
    About me: this piece of Code checks if the shakespeare data performs good with my code.
    Problem: The problem we had in the previous (learn-4 file is thay the loss we encountered was "nan")
    Approach: Here we take the Shakespeare data and check if the shakespeare data performs good with my code.
                if the output here is good we conclude that the code is okay and the data is not proper.
    
    Needed: Here to run this code we need the shakespeare data in the local directory. IF Not there then refer learn-3
    Output: we see from the output that the loss in this case is not "nan", which means that the code is correct.
"""
import numpy as np
import time
import os
import urllib.request
from six.moves import cPickle as pickle


import tensorflow as tf
from tensorflow.models.rnn.ptb import reader
from gensim import corpora

import matplotlib.pyplot as plt

In [8]:
file_name = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/Practise/tinyshakespeare.txt'
with open(file_name,'r') as f:
    raw_data = f.read()
    print("Data length:", len(raw_data))
    
vocab = set(raw_data)
vocab_size = len(vocab)
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))

data = [vocab_to_idx[c] for c in raw_data]
del raw_data

Data length: 1115394


In [9]:
def reset_graph():  # Reset the graph
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [21]:
def dynamic_RNN_model(
    num_hid_units = 100,
    momentum = 0.9,
    num_classes = 10,
    learning_rate = 0.01
    ):
    vocab_size = num_classes
    print ('The num of hidden unit is: ', num_hid_units)
    print ('The Vocab size is: ', vocab_size)
    print ('The momentum is: ', momentum)
    print ('The learning_rate is: ', learning_rate)
    
    
    reset_graph()
    
    x = tf.placeholder(tf.int32, shape = [None, None], name='input_placeholder')
    y = tf.placeholder(tf.int32, shape = [None, None], name='output_placeholdr')
    batch_size = tf.shape(x)[0]
    

    # ENBEDDING(INPUT) LAYER OPERATION
    # Creating an Embedding matrix with a random weight for all vacab to hidden_matrix
    embed_to_hid_wghts = tf.get_variable('embedding_matrix', [vocab_size, num_hid_units])
    embed_to_hid_layer = tf.nn.embedding_lookup(embed_to_hid_wghts, x)
    print ('The shape of embed_to_hid_wghts is: ', embed_to_hid_wghts.get_shape())
    print ('The shape of embed_to_hid_layer is: ', embed_to_hid_layer.get_shape())


    # HIDDEN LAYER OPERATION
    rnn_cell = tf.nn.rnn_cell.LSTMCell(num_hid_units, state_is_tuple=True)
    init_state = rnn_cell.zero_state(batch_size, tf.float32)  # Each sequence will hava a state that it passes to its next sequence
    rnn_outputs, new_state = tf.nn.dynamic_rnn(cell=rnn_cell,
                                            # sequence_length=X_lengths,
                                            initial_state=init_state,
                                            inputs=embed_to_hid_layer)
    

    # OUTPUT LAYER OPERATION
    # Initialize the weight and biases for the output layer. We use variable scope because we would like to share the weights 
    with tf.variable_scope('output_layer'):
        hid_to_output_wght = tf.get_variable('hid_to_output_wght', [num_hid_units, num_classes])
        output_bias = tf.get_variable('output_bias',
                                      [num_classes],
                                      initializer=tf.constant_initializer(0.0))
    
    rnn_outputs = tf.reshape(rnn_outputs, [-1, num_hid_units])  
    hid_to_ouptut_layer = tf.matmul(rnn_outputs, hid_to_output_wght) +  output_bias
    output_state = tf.nn.softmax(hid_to_ouptut_layer, name=None)
 
    
    
    # CALCULATING LOSS, OPTIMIZING THE COST FUNCTION, MEASURING ACCURACY
    y_reshaped = tf.reshape(y, [-1])
    loss_CE = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(hid_to_ouptut_layer, y_reshaped))
    
    # The sparse_softmax uses dtype as int32 or int64
    optimizer = tf.train.MomentumOptimizer(learning_rate, 
                                            momentum, 
                                            use_locking=False, 
                                            name='Momentum', 
                                            use_nesterov=True).minimize(loss_CE)
    # y_ = tf.reshape(y, [-1])
    # correct_prediction = tf.equal(tf.arg_max(output_state,1), tf.arg_max(y_ ,1))
    # accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    

    # Returns graph objects
    return dict(
        x=x,
        y=y,
        batch_size = batch_size,
        init_state = init_state,
        new_state = new_state,
        loss_CE = loss_CE,
        optimizer = optimizer,
        training_prediction = output_state
    )

In [33]:
def gen_epochs(n, num_steps, batch_size):
    for i in range(n):
        yield reader.ptb_iterator(data, batch_size, num_steps)

def train_network(g, num_epochs, num_steps = 200, batch_size = 32, verbose = True, save=False):
    tf.set_random_seed(2345)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps, batch_size)):
            training_loss = 0
            steps = 0
            new_hid_layer_state = None
            
            for X, Y in epoch:
                if not new_hid_layer_state:
                    print ('Using the zero init RNN State')
                    feed_dict= {graph_dict['x']: X, 
                                graph_dict['y']: Y}
                else:
                    print ('Using the new RNN State')
                    feed_dict= {graph_dict['x']: X, 
                                graph_dict['y']: Y,
                                graph_dict['init_state'] : new_hid_layer_state}

                bs, nwst, loss, opt, tp = sess.run([graph_dict['batch_size'],
                                                graph_dict['new_state'],
                                                graph_dict['loss_CE'],
                                                graph_dict['optimizer'],
                                                graph_dict['training_prediction']], 
                                                feed_dict=feed_dict)
                new_hid_layer_state = nwst
                training_loss += loss
#                 acc = self.accuracy(tp, batch_train_labels)

#                 print ('accuracy of the batch %d is: '%no, acc)
#                 print ('')
                print ('Average Loss for the batch is:', loss)
                print ('')


In [34]:
t = time.time()
dynamic_RNN_model()
print("It took", time.time() - t, "seconds to build the graph.")

The num of hidden unit is:  100
The Vocab size is:  10
The momentum is:  0.9
The learning_rate is:  0.01
The shape of embed_to_hid_wghts is:  (10, 100)
The shape of embed_to_hid_layer is:  (?, ?, 100)
It took 0.2942180633544922 seconds to build the graph.


In [35]:
print ('vocab_size is: ', vocab_size)
graph_dict =  dynamic_RNN_model(num_hid_units = 100, num_classes = vocab_size)
t = time.time()
train_network(graph_dict, num_epochs=1, num_steps = 200, batch_size = 32)
print("It took", time.time() - t, "seconds to train for 1 epochs.")

vocab_size is:  65
The num of hidden unit is:  100
The Vocab size is:  65
The momentum is:  0.9
The learning_rate is:  0.01
The shape of embed_to_hid_wghts is:  (65, 100)
The shape of embed_to_hid_layer is:  (?, ?, 100)
Using the zero init RNN State
Average Loss for the batch is: 4.17213

Using the new RNN State
Average Loss for the batch is: 4.16948

Using the new RNN State
Average Loss for the batch is: 4.16546

Using the new RNN State
Average Loss for the batch is: 4.16034

Using the new RNN State
Average Loss for the batch is: 4.15652

Using the new RNN State
Average Loss for the batch is: 4.1506

Using the new RNN State
Average Loss for the batch is: 4.1449

Using the new RNN State
Average Loss for the batch is: 4.13654

Using the new RNN State
Average Loss for the batch is: 4.12921

Using the new RNN State
Average Loss for the batch is: 4.12009

Using the new RNN State
Average Loss for the batch is: 4.11362

Using the new RNN State
Average Loss for the batch is: 4.10106

Using th